In [171]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [183]:
import sys
sys.path.append("../../src") 

import pickle

import pandas as pd
import numpy as np
import networkx as nx
import pymc3 as pm

from itertools import combinations
from tqdm import tqdm

import MCMC_Edges as mcmc

In [174]:
LUCAS_NODES = ('Smoking', 'Lung_cancer', 'Genetics', 'Attention_Disorder', 'Yellow_Fingers', 'Anxiety', 'Peer_Pressure', 'Allergy', 'Coughing', 'Fatigue', 'Car_Accident', 'Born_an_Even_Day')
POSSIBLE_EDGES = [*combinations(LUCAS_NODES, 2)]

In [175]:
f = open("final_population.txt", "r")
networks = []

for i, x in enumerate(f):
    edges_string = x.split(';')[:-1]
    network = []
    
    for edge_str in edges_string:
        network\
            .append(
                list(
                    edge_str[1:-1]\
                        .replace("'", "")\
                        .replace(' ', '')\
                        .split(',')
                )
            )
    
    networks.append(nx.DiGraph())
    networks[i].add_nodes_from(LUCAS_NODES)
    networks[i].add_edges_from(network)

print(str(len(networks)) + ' networks')

605 networks


In [176]:
observed_networks = [[] for i in range(len(networks))]

j = 0
for i, network in enumerate(networks):
    observed_networks[i].append([])

    for edge in POSSIBLE_EDGES:
        if (edge in network.edges):
            observed_networks[i][j].append(-1)
        elif ((edge[1], edge[0]) in network.edges):
            observed_networks[i][j].append(1)
        else: 
            observed_networks[i][j].append(0)

observed_networks = [np.array(x) for x in observed_networks]
observed_networks = np.array(observed_networks)

In [177]:
observed_networks[600][0]

array([ 1,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [178]:
values = [-1, 0, 1]
k = len(values)          
n = len(networks)
total_count = len(observed_networks[0])

In [184]:
def learn_edge(edge, edge_index, chains=10, tune=10000, draws=10000, nthreads=10):
    observed_edges = mcmc.get_edge_frequency(values, observed_networks, edge_index)

    with pm.Model() as model_dm_explicit:
        frac = pm.Dirichlet("frac", a=np.ones(k))
        conc = pm.Lognormal("conc", mu=1, sigma=1)
        p = pm.Dirichlet("p", a=frac * conc, shape=(n, k))

        counts = pm.Multinomial("counts", n=total_count, p=p, shape=(n, k), observed=observed_edges)

        trace_dm_explicit = pm.sample(chains=chains, tune=tune, draws=draws, step=pm.NUTS(), return_inferencedata=False, cores=nthreads)
        
        with open('./mcmc/pfp_v1_mcmc_' + edge[0]+"-"+edge[1] + '.pickle', 'wb') as handle:
            pickle.dump({
                'model': model_dm_explicit, 
                'frac':  frac, 
                'conc': conc, 
                'p': p, 
                'counts': counts, 
                'trace': trace_dm_explicit
            }, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        return pm.summary(trace_dm_explicit)

In [ ]:
models = dict()

for i, edge in enumerate(tqdm(POSSIBLE_EDGES)):
    models[edge[0]+"-"+edge[1]] = learn_edge(edge, i, chains=10, tune=1500, draws=1500, nthreads=20)
    
    clear_output(wait=True)

  0%|                                                    | 0/66 [00:00<?, ?it/s]Multiprocess sampling (10 chains in 20 jobs)
NUTS: [p, conc, frac]
